In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.decomposition import PCA
import imageio
import random
import time
from itertools import permutations
from matplotlib import animation
from JSAnimation.IPython_display import display_animation
from sklearn.cluster import AgglomerativeClustering
from IPython.display import clear_output
import seaborn as sns; sns.set()
from scipy.stats import entropy
#from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.preprocessing import StandardScaler
#from skimage import img_as_ubyte

In [2]:
def sig_func(x,y):
    
    """Example function with PEP 484 type annotations.

    Args:
        x: The first parameter.
        y: The second parameter.

    Returns:
        The return value.
        
    """
    
    return(1/(1+np.exp(-(np.dot(x,y)))))

In [3]:
def mutate(x,rate):
    
    """Example function with PEP 484 type annotations.

    Args:
        x: The first parameter.
        rate: The second parameter.

    Returns:
        The return value.
        
    """
    
    chosen = np.random.choice(range(0,len(x)),rate,replace=False)
    for i in range(0,len(x)):
        if i in chosen:
            x[i] = 2*np.random.random(1)-1
    return(x)

In [4]:
def intersection(lst1, lst2):
    
    """Example function with PEP 484 type annotations.

    Args:
        lst1: The first parameter.
        lst2: The second parameter.

    Returns:
        The return value.
        
    """
    
    return list(set(lst1) & set(lst2))

In [ ]:
def extract_tribe(cells, cell_weights, n, labels, node_n = 225):
    
    """Example function with PEP 484 type annotations.

    Args:
        lst1: The first parameter.
        lst2: The second parameter.

    Returns:
        The return value.
        
    """
    
    db = AgglomerativeClustering(n).fit(cell_weights)
    print(db.labels_)
    flags = []
    for k in labels:
        flags = [flags, [j for j, x in enumerate(db.labels_ == k) if x]]
    flags = flags[1:]
    print(flags)
    next_cells = cells[flags,:]  
    next_weights = cell_weights[flags,:]
    return(next_cells[0], next_weights[0], db)

In [ ]:
def fuzzy_logic(a):
    
    """Example function with PEP 484 type annotations.

    Args:
        lst1: The first parameter.
        lst2: The second parameter.

    Returns:
        The return value.
        
    """
    
    less_norm = np.linalg.norm([i for i in a if i <= np.median(a)])
    cutoff = np.min(a)/less_norm + np.random.rand()*((np.median(a)-np.min(a))/less_norm)
    choice = a.index(np.random.choice([i for i in a if i <= cutoff*less_norm]))
    
    return(choice)

In [ ]:
def slime(x = [],
          y = [],
          start_pop = 200,
          max_age = 100,
          fig_size = [5,5],
          board_size = 128,
          fertile = 6,
          start_hp = 4,
          log = False,
          frames=500,
          interval=50,
          mutation_rate = 40,
          willing_wait = 5,
          experimental = False,
          mode='once'):
    
    """Example function with PEP 484 type annotations.

    Args:
        lst1: The first parameter.
        lst2: The second parameter.

    Returns:
        The return value.

    """
    
    global sight
    global time_tick
    global grow_count
    global birth_count
    global bump_count
    global trends
    global cells
    global cell_weights
    global cell_name
    global killed
    global border
    global last_time
    global new_coord
    global G
    global for_gif
    global max_hp
    global frame_count
    
    total_start_time = time.time()
    frame_count = 1
    max_hp = fertile
    sight = 9
    G = nx.Graph()
    time_tick = 0
    trends = np.array([0,0,0,0,0,0,0])
    border = round(board_size/3)
    last_time = 0
    cell_name = 1
    new_coord = [[-1,1],
                 [0,1],
                 [1,1],
                 [-1,0],
                 [0,0],
                 [1,0],
                 [-1,-1],
                 [0,-1],
                 [1,-1]]
    
    # Build the initial cell list.  Skip this if one from a previous run is submitted.
    # The NN is built around sight/input of 9*9, movement/output of 3*3.
    if(len(x) < 1):
        cells = np.zeros((start_pop,5))
        cell_weights = np.zeros((start_pop,(sight*sight*3*3)))
        
        for cell in range(0,start_pop):
            cells[cell,:] = [cell_name,
                          np.random.choice(range(border, board_size-border)),
                          np.random.choice(range(border, board_size-border)),
                          start_hp,
                          1]
            cell_weights[cell,:] = 2*np.random.random((sight*sight*3*3))-1
            G.add_node(cell)
            cell_name += 1          
    else:
        cells = x
        cell_weights = y

    # Create the board.
    X = np.zeros(shape=(board_size,board_size))
    for_gif = X
    
    # initialization function: plot the background of each frame
    def init():
        im.set_data(10*X)
        return(im,)
    
    # Get the relative coodinates for the input.
    L = [0, 0, 1, 1, -1, -1, 2, 2, -2, -2, 3, 3, -3, -3, 4, 4, -4, -4]
    keep = [0,0]
    
    for comb in permutations(L, r=2):
        keep = np.vstack((keep, comb))
    
    # Setup plotting.
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
    im = ax.imshow(X, cmap='magma', interpolation='nearest')
    im.set_clim(0, 8)
        
    def animate(i):
        global sight
        global time_tick
        global grow_count
        global birth_count
        global bump_count
        global trends
        global cells
        global cell_weights
        global cell_name
        global total_pop
        global killed
        global border
        global last_time
        global new_coord
        global G
        global for_gif
        global max_hp
        global frame_count
        
        """Example function with PEP 484 type annotations.

        Args:
            lst1: The first parameter.
            lst2: The second parameter.

        Returns:
            The return value.

        """
        
        # Warning for long frame generation wait time.
        if(last_time > willing_wait):
            print("Taking too long...  Frame {0} took {1} seconds.".format((frame_count-1),round(last_time,1)))
        
        # Initial values.
        start_time = time.time()
        grow_count = 0
        birth_count = 0
        bump_count = 0
        killed_this_round = np.array([], dtype = int)
        
        # Take turns through the cell list for each frame.
        time_tick += 1
        im.set_data(10*animate.X)
        
        try:
            p = (np.transpose(PCA(n_components=3).fit(np.transpose(cell_weights)).components_)+1)/2
            rgb = np.round(255*(p-np.min(p))/(np.max(p)-np.min(p)))/((256^2)*256 + 256*256 + 256)
            cell_color = (256^2)*rgb[:,0] + 256*rgb[:,1] + rgb[:,2]
            
        except:
            print("Extinction event occured!  Had to return")
            return(im,)
        
        
        for cell in range(0, len(cells)):
            
            # Input.
            test_in = []
            for perm in np.mod(cells[cell,1:3] + np.unique(keep,axis=0), board_size):
                test_in = np.append(test_in, X[int(perm[0]), int(perm[1])])

            # Output.
            test_out = sig_func(test_in,cell_weights[cell,:].reshape((sight*sight, 3*3)))
            d = fuzzy_logic(test_out.tolist())

            # Rest and recover up to half fertile health.
            if(d==4):
                if(cells[cell,3] < max_hp/2):
                    cells[cell,3] += 1
                    grow_count += 1
                    if(log == True):
                        print("{0} grew to size {1}.".format(cells[cell,0], cells[cell,3]))

                # Handle birth/splitting, don't clear.
                if(cells[cell,3] >= max_hp):
                    
                    # Get the relative coodinates for possible baby locations.
                    L = [0, 0, 1, 1, -1, -1]
                    keep2 = [0,0]

                    for comb in permutations(L, r=2):
                        keep2 = np.vstack((keep2, comb))

                    potential = []
                    for perm in np.mod(cells[cell,1:3] + np.unique(keep2,axis=0), board_size):
                        potential = np.append(potential, X[int(perm[0]), int(perm[1])])
                    
                    # Check if location for baby is clear
                    clear = []
                    for i in potential:
                        if(i == 0):
                            clear = np.append(clear,i)
                    
                    # Cell division process, only divide if there is a vacant destination to choose from.
                    if(len(clear) > 0):
                        d = np.random.choice(range(0,len(clear)))
                        # Give new coordinate and fill it in.
                        destination = np.mod(np.add(cells[int(cell),1:3], new_coord[d]), board_size)
                        
                        # Half parent hp.
                        divided_hp = np.floor(cells[cell,3]/2)
                        cells[cell,3] = divided_hp

                        # Give the baby attributes.
                        baby_name = cell_name
                        G.add_node(baby_name)
                        G.add_edge(cell_name, baby_name)
                        cell_name += 1

                        a = [baby_name,
                             int(destination[0]),
                             int(destination[1]),
                             divided_hp,
                             1]
                        
                        # Mutate the baby's NN
                        b = mutate(cell_weights[cell,:], mutation_rate)
                        cells = np.vstack((cells, a)) 
                        cell_weights = np.vstack((cell_weights, b))
                        #X[int(destination[0]), int(destination[1])] = abs(sum(b))
                        p = (np.transpose(PCA(n_components=3).fit(np.transpose(cell_weights)).components_)+1)/2
                        rgb = np.round(255*(p-np.min(p))/(np.max(p)-np.min(p)))/((256^2)*256 + 256*256 + 256)
                        X[int(destination[0]), int(destination[1])] = (256^2)*rgb[cell,0] + 256*rgb[cell,1] + rgb[cell,2]

                        birth_count += 1
                        if(log == True):
                            print("{0} underwent mitosis.  Gave birth to {1}.".format(cells[cell,0], baby_name))

            # Movement logic.
            else:
                # If it is clear to move.
                destination = np.mod(np.add(cells[cell,1:3], new_coord[d]), board_size)
                
                if(X[int(destination[0]), int(destination[1])] == 0):
                    # Clear where it was since it is not splitting.
                    last_spot = cells[cell,1:3]
                    X[int(last_spot[0]),int(last_spot[1])] = 0         
                    # Give new coordinate and fill it in.
                    cells[cell,1:3] = destination
                    #X[int(destination[0]), int(destination[1])] = abs(sum(cell_weights[cell,:]))
                    X[int(destination[0]), int(destination[1])] = cell_color[cell]
                    
                else:
                    # It has an altercation    
                    x = [i for i, j in enumerate(cells[:,1]) if j == destination[0]]
                    y = [i for i, j in enumerate(cells[:,2]) if j == destination[1]]
                    z = intersection(x,y)
                    
                    # Make sure there is no ambiguity
                    if(len(z) == 1):
                        # The outcome is determined by hit points, age and who is initiating.
                        # The following ratio incorperates ages of predator and prey
                        rat = np.log(cells[z,4]/cells[cell,4])/np.e
                        
                        # Experimental ################################################
                        if(experimental == True):
                            # Get the relative coodinates
                            L = [0, 0, 1, 1, -1, -1]
                            keep2 = [0,0]
                            for comb in permutations(L, r=2):
                                keep2 = np.vstack((keep2, comb))
                            potential = []
                            for perm in np.mod(cells[cell,1:3] + np.unique(keep2,axis=0), board_size):
                                potential = np.append(potential, X[int(perm[0]), int(perm[1])])
                        ###############################################################
                        
                        if(cells[cell,3] > cells[z,3]):
                            attacker_hp_before = cells[cell,3]
                            defender_hp_before = cells[z,3]
                            cells[cell,3] += np.round(3+rat)
                            cells[z,3] -= np.round(3+rat)
                            
                            if(log == True):
                                print("{0} attacked {1}!".format(cells[cell,0], cells[z,0]))
                                print("{0} hp: {1} --> {2}".format(cells[cell,0], attacker_hp_before, cells[cell,3]))
                                print("{0} hp: {1} --> {2}".format(cells[z,0], defender_hp_before, cells[z,3]))

                            
                            # Experimental ################################################
                            if(experimental == True):
                                reward = np.array((np.repeat(1,9)))
                                reward[d] = 0
                                cell_weights[cell,:] += np.dot(potential.T,(test_out-reward)*reward*(1-reward))     
                            ###############################################################
                            
                        if(cells[cell,3] == cells[z,3]):
                            attacker_hp_before = cells[cell,3]
                            defender_hp_before = cells[z,3]
                            cells[cell,3] += np.round(1+rat)
                            cells[z,3] -= np.round(1+rat)
                            
                            if(log == True):
                                print("{0} attacked {1}!".format(cells[cell,0], cells[z,0]))
                                print("{0} hp: {1} --> {2}".format(cells[cell,0], attacker_hp_before, cells[cell,3]))
                                print("{0} hp: {1} --> {2}".format(cells[z,0], defender_hp_before, cells[z,3]))
                            
                            # Experimental ################################################
                            if(experimental == True):
                                reward = np.array((np.repeat(1,9)))
                                reward[d] = 0
                                cell_weights[cell,:] += np.dot(potential.T,(test_out-reward)*reward*(1-reward))
                            ###############################################################
                                
                        if(cells[cell,3] < cells[z,3]):
                            attacker_hp_before = cells[cell,3]
                            defender_hp_before = cells[z,3]
                            cells[cell,3] -= np.round(2-rat)
                            cells[z,3] += np.round(2-rat)
                            
                            if(log == True):
                                print("{0} attacked {1}!".format(cells[cell,0], cells[z,0]))
                                print("{0} hp: {1} --> {2}".format(cells[cell,0], attacker_hp_before, cells[cell,3]))
                                print("{0} hp: {1} --> {2}".format(cells[z,0], defender_hp_before, cells[z,3]))
                                
                            # Experimental ################################################
                            if(experimental == True):
                                reward = np.array((np.repeat(0,9)))
                                reward[d] = 1
                                cell_weights[cell,:] += np.dot(potential.T,(test_out-reward)*reward*(1-reward)) 
                            ###############################################################
                            
                        # Experimental ################################################
                        if(log == True):
                            print("Experimental nurture reenforcement modified {0}'s neural network.".format(cells[cell,0]))  
                        ###############################################################
                            
                        bump_count += 1
                            
                        try:
                            # Handle cell death.
                            if(cells[cell,3] < 1):
                                # Clear where the attacker was and add to the death list.
                                last_spot = cells[cell,1:3]
                                X[int(last_spot[0]),int(last_spot[1])] = 0
                                killed_this_round = np.append(killed_this_round,int(cells[cell,0]))
                                if(log == True):
                                    print("{0} died!.".format(cells[cell,0]))

                            if(cells[z,3] < 1):
                                # Clear where the victim was and add to the death list.
                                last_spot = cells[z,1:3]
                                X[int(last_spot[0]),int(last_spot[1])] = 0
                                killed_this_round = np.append(killed_this_round,int(cells[z,0]))
                                if(log == True):
                                    print("{0} died!.".format(cells[z,0]))
                        except:
                            pass
        
        # Kill the elderly        
        for i in range(0,len(cells)):
            if(cells[int(i),4] > max_age):
                # Blur max age a bit.
                if(np.random.random() < (cells[int(i),4]-max_age)/max_age):      
                    # Clear where it was and add to the death list.
                    last_spot = cells[cell,1:3]
                    X[int(last_spot[0]),int(last_spot[1])] = 0
                    killed_this_round = np.append(killed_this_round,int(cells[cell,0]))
                    if(log == True):
                        print("{0} died of old age!.".format(cells[cell,0]))
        
        # Finally handle the death list and delete cells
        for i in np.unique(killed_this_round):
            j = list(cells[:,0]).index((i))
            cells = np.delete(cells, (j), axis=0)
            cell_weights = np.delete(cell_weights, (j), axis=0)
        
        # Dynamic population control
        run_pop = len(cells)   
        if(np.random.random() < -np.log(run_pop/(start_pop)/np.e)):
            if(max_hp > 3):
                    max_hp -= 1
                    print("We're going extinct, make babies!  Breeding requirements decreased to {0}.".format(max_hp))

        if(np.random.random() < np.log(run_pop/(start_pop)/np.e)):
                max_hp += 1
                print("Too crowded, stop having kids!  Breeding requirements increased to {0}.".format(max_hp))
        
        mean_var = np.mean(np.var(cell_weights,axis = 0))
        
        # Tick everyone's age
        cells[:,4] = np.add(cells[:,4],1)
        # Capture array for gif
        for_gif = np.concatenate((for_gif, X))
        # Capture death statistics
        death_count = len(np.unique(killed_this_round))
        # Current frame summary
        current = np.array([time_tick, grow_count, birth_count, bump_count, death_count, run_pop, mean_var])
        # Append to run summary
        trends = np.vstack((trends, current))
        # Get the current frame run time to check next frame
        last_time = time.time() - start_time
        # Log every 10 frames to monitor full run time and check frame generation time.
        if(np.mod(frame_count,10) == 0):
            clear_output(wait=True)
            print("Frame {0} took {1} seconds.".format(frame_count,round(last_time,1)))
        frame_count += 1
    
        return(im,)
    
    # Handles the browser animation viewer.
    animate.X = X
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, interval=interval, blit=True)
    html_result = display_animation(anim, default_mode=mode)
    
    # Make a little animated gif
    images = []
    for i in range(0,(frames+1)):
        converted = np.reshape(for_gif,(len(trends),board_size,board_size))[int(i),:,:]
        imageio.imwrite('slime.gif', converted)
        clear_output(wait=True)
        images.append(imageio.imread('slime.gif'))
    imageio.mimsave('slime.gif', images)
    
    # Total run time
    total_last_time = time.time() - total_start_time
    clear_output(wait=True)
    print("Total Run Time:  {0} seconds.".format(round(total_last_time,1)))
    
    return(html_result, cells, cell_weights, G, trends[1:,:], for_gif)

In [ ]:
gr, c, cw, g, t, a = slime(start_pop=100, board_size=128, frames=100, experimental = True, log = True)

Frame 90 took 2.1 seconds.
77.0 grew to size 1.0.
85.0 grew to size 0.0.
129.0 grew to size 5.0.
148.0 attacked [800.]!
148.0 hp: 0.0 --> -3.0
[800.] hp: [6.] --> [9.]
Experimental nurture reenforcement modified 148.0's neural network.
148.0 died!.
150.0 attacked [727.]!
150.0 hp: 23.0 --> 25.0
[727.] hp: [21.] --> [19.]
Experimental nurture reenforcement modified 150.0's neural network.
152.0 attacked [113.]!
152.0 hp: 4.0 --> 2.0
[113.] hp: [27.] --> [29.]
Experimental nurture reenforcement modified 152.0's neural network.
155.0 attacked [85.]!
155.0 hp: 11.0 --> 14.0
[85.] hp: [0.] --> [-3.]
Experimental nurture reenforcement modified 155.0's neural network.
195.0 grew to size 2.0.
296.0 grew to size 5.0.
316.0 grew to size 5.0.
341.0 grew to size -1.0.
399.0 grew to size 5.0.
415.0 grew to size 5.0.
473.0 grew to size 3.0.
497.0 underwent mitosis.  Gave birth to 807.
502.0 attacked [681.]!
502.0 hp: 9.0 --> 12.0
[681.] hp: [7.] --> [4.]
Experimental nurture reenforcement modified 5

745.0 underwent mitosis.  Gave birth to 816.
753.0 grew to size 5.0.
757.0 grew to size 4.0.
760.0 grew to size 4.0.
764.0 attacked [649.]!
764.0 hp: 7.0 --> 10.0
[649.] hp: [0.] --> [-3.]
Experimental nurture reenforcement modified 764.0's neural network.
775.0 attacked [355.]!
775.0 hp: 5.0 --> 7.0
[355.] hp: [5.] --> [3.]
Experimental nurture reenforcement modified 775.0's neural network.
781.0 grew to size 5.0.
793.0 attacked [771.]!
793.0 hp: 3.0 --> 1.0
[771.] hp: [13.] --> [15.]
Experimental nurture reenforcement modified 793.0's neural network.
794.0 attacked [629.]!
794.0 hp: -1.0 --> -2.0
[629.] hp: [5.] --> [6.]
Experimental nurture reenforcement modified 794.0's neural network.
794.0 died!.
797.0 attacked [36.]!
797.0 hp: 5.0 --> 7.0
[36.] hp: [5.] --> [3.]
Experimental nurture reenforcement modified 797.0's neural network.
798.0 attacked [14.]!
798.0 hp: 13.0 --> 17.0
[14.] hp: [-17.] --> [-21.]
Experimental nurture reenforcement modified 798.0's neural network.
800.0 atta

14.0 grew to size -19.0.
25.0 attacked [477.]!
25.0 hp: 4.0 --> 2.0
[477.] hp: [5.] --> [7.]
Experimental nurture reenforcement modified 25.0's neural network.
36.0 grew to size 4.0.
87.0 attacked [806.]!
87.0 hp: 24.0 --> 26.0
[806.] hp: [5.] --> [3.]
Experimental nurture reenforcement modified 87.0's neural network.
137.0 grew to size 2.0.
277.0 grew to size 5.0.
332.0 attacked [212.]!
332.0 hp: 5.0 --> 3.0
[212.] hp: [6.] --> [8.]
Experimental nurture reenforcement modified 332.0's neural network.
355.0 attacked [782.]!
355.0 hp: 3.0 --> 3.0
[782.] hp: [3.] --> [3.]
Experimental nurture reenforcement modified 355.0's neural network.
391.0 underwent mitosis.  Gave birth to 824.
461.0 grew to size 1.0.
477.0 attacked [25.]!
477.0 hp: 7.0 --> 10.0
[25.] hp: [2.] --> [-1.]
Experimental nurture reenforcement modified 477.0's neural network.
521.0 grew to size 5.0.
535.0 attacked [809.]!
535.0 hp: 1.0 --> -2.0
[809.] hp: [22.] --> [25.]
Experimental nurture reenforcement modified 535.0's 

In [ ]:
gr

In [ ]:
# Growth
plt.plot(t[:,0],t[:,1])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Growth')
plt.grid(True)
plt.show()

# Birth
plt.plot(t[:,0],t[:,2])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Birth')
plt.grid(True)
plt.show()

# Injuries
plt.plot(t[:,0],t[:,3])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Injuries')
plt.grid(True)
plt.show()

# Deaths
plt.plot(t[:,0],t[:,4])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Deeaths')
plt.grid(True)
plt.show()

# Births-Deaths
plt.plot(t[:,0],t[:,2]-t[:,4])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Births - Deaths')
plt.grid(True)
plt.show()

# Population
plt.plot(t[:,0],t[:,5])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Population')
plt.grid(True)
plt.show()

# Mean Variance
plt.plot(t[:,0],t[:,6])
plt.xlabel('Time (frame)')
plt.ylabel('Value')
plt.title('Mean Variance')
plt.grid(True)
plt.show()

In [ ]:
# Summary
t

In [ ]:
nx.info(g)

***
# Continue with previous results
***

In [ ]:
c, cw, db = extract_tribe(c, cw, 5, [0])

In [ ]:
ax = sns.heatmap(cw[0].reshape((sight*sight, 3*3)))

In [ ]:
entropy(cw[0])

In [ ]:
gr, c, cw, g, t, a = slime(c[0:100,:], cw[0:100,:], board_size=128, start_pop=100, experimental = True, frames=1000)

In [ ]:
gr

In [ ]:
# Growth
plt.plot(t[:,0],t[:,1])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Growth')
plt.grid(True)
plt.show()

# Birth
plt.plot(t[:,0],t[:,2])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Birth')
plt.grid(True)
plt.show()

# Injuries
plt.plot(t[:,0],t[:,3])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Injuries')
plt.grid(True)
plt.show()

# Deaths
plt.plot(t[:,0],t[:,4])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Deeaths')
plt.grid(True)
plt.show()

# Births-Deaths
plt.plot(t[:,0],t[:,2]-t[:,4])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Births - Deaths')
plt.grid(True)
plt.show()

# Population
plt.plot(t[:,0],t[:,5])
plt.xlabel('Time (frame)')
plt.ylabel('Count')
plt.title('Population')
plt.grid(True)
plt.show()

# Mean Variance
plt.plot(t[:,0],t[:,6])
plt.xlabel('Time (frame)')
plt.ylabel('Value')
plt.title('Mean Variance')
plt.grid(True)
plt.show()

In [ ]:
# Summary
t

In [ ]:
nx.info(g)

In [ ]:
#import dill
#dill.dump_session('notebook_env.db')

In [ ]:
#import dill
#dill.load_session('notebook_env.db')

In [ ]:
#for i in range(1):
#    output = sig_func(train_in,disc_weights)
#    disc_weights += np.dot(train_in.T,(train_out-output)*output*(1-output))

#test_in = animate.X[coords[0]-lookr:coords[0]+lookr+1,coords[1]-lookr:coords[1]+lookr+1].flatten()
#test_out = sig_func(test_in,disc_weights)
#train_in = np.vstack((train_in,test_in))

#animate.X = life_step(animate.X)
#outcome = animate.X[coords[0]-1:coords[0]+2,coords[1]-1:coords[1]+2].flatten()
#train_out = np.vstack((train_out,outcome))